**Examples of Collaborative Filtering based Recommendation Systems**

In [1]:
#make necesarry imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
import sys, os
from contextlib import contextmanager

In [2]:
r = pd.read_csv( 'ratings.csv' ) #file full of ratings
tr = pd.read_csv( 'to_read.csv' ) #bookss to read by user id
b = pd.read_csv( 'books.csv' )

t = pd.read_csv( 'tags.csv' )
bt = pd.read_csv( 'book_tags.csv')


In [3]:
t.loc[0]

tag_id      0
tag_name    -
Name: 0, dtype: object

In [4]:
d = r.pivot(index='user_id', columns='book_id', values='rating')
d = d.fillna(0)
M=d[:1000]

In [5]:
M.tail()

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
996,0.0,3.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
global k,metric
k=4
metric='cosine' #can be changed to 'correlation' for Pearson correlation similaries

**User-based Recommendation Systems**

In [12]:
#get cosine similarities for ratings matrix M; pairwise_distances returns the distances between ratings and hence
#similarities are obtained by subtracting distances from 1
cosine_sim = 1-pairwise_distances(M, metric="cosine")

In [18]:
#Cosine similarity matrix
pd.DataFrame(cosine_sim).head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.084493,0.078952,0.232851,0.014549,0.067218,0.045614,0.126093,0.185741,0.232423,...,0.092105,0.091808,0.166394,0.139215,0.155263,0.115535,0.197652,0.084747,0.040299,0.098220
1,0.084493,1.000000,0.047477,0.213506,0.000000,0.040890,0.035401,0.069531,0.154285,0.112504,...,0.162230,0.137646,0.054521,0.132961,0.065168,0.079987,0.094070,0.007702,0.014893,0.062653
2,0.078952,0.047477,1.000000,0.125132,0.000000,0.042397,0.071152,0.123920,0.086424,0.079583,...,0.055023,0.069851,0.108678,0.076577,0.086230,0.054197,0.117655,0.007502,0.030173,0.072649
3,0.232851,0.213506,0.125132,1.000000,0.004859,0.048208,0.140826,0.149132,0.223924,0.190329,...,0.232117,0.230883,0.166518,0.188526,0.167409,0.165746,0.193626,0.032169,0.094269,0.149182
4,0.014549,0.000000,0.000000,0.004859,1.000000,0.000000,0.000000,0.007833,0.005259,0.004437,...,0.015339,0.000000,0.038043,0.000000,0.008681,0.000000,0.009433,0.054372,0.000000,0.003949


In [14]:
#get pearson similarities for ratings matrix M
pearson_sim = 1-pairwise_distances(M, metric="correlation")

In [17]:
#Pearson correlation similarity matrix
pd.DataFrame(pearson_sim).head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.076874,0.070713,0.223773,0.004307,0.058067,0.033438,0.117444,0.176343,0.223552,...,0.083309,0.080877,0.157348,0.129168,0.146055,0.104436,0.187028,0.075526,0.030055,0.088824
1,0.076874,1.000000,0.041026,0.206583,-0.007856,0.033779,0.026126,0.062572,0.146976,0.104801,...,0.156104,0.129882,0.046781,0.125375,0.057485,0.071315,0.085067,0.000143,0.006949,0.055283
2,0.070713,0.041026,1.000000,0.116777,-0.008429,0.034776,0.061583,0.116888,0.077897,0.070978,...,0.047593,0.060798,0.100851,0.067859,0.078166,0.044593,0.108239,-0.000611,0.021782,0.064819
3,0.223773,0.206583,0.116777,1.000000,-0.006225,0.038205,0.129079,0.140126,0.214322,0.180299,...,0.224186,0.220960,0.156832,0.178373,0.157690,0.154518,0.182153,0.021720,0.083914,0.139693
4,0.004307,-0.007856,-0.008429,-0.006225,1.000000,-0.009231,-0.012005,-0.001410,-0.005542,-0.006388,...,0.006359,-0.011324,0.028225,-0.010982,-0.001483,-0.011816,-0.002952,0.045408,-0.010040,-0.005813


In [20]:
#This function finds k similar users given the user_id and ratings matrix M
#Note that the similarities are same as obtained via using pairwise_distances
def findksimilarusers(user_id, ratings, metric = metric, k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.iloc[user_id-1, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    print ('{0} most similar users for User {1}:\n'.format(k,user_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;

        else:
            print ('{0}: User {1}, with similarity of {2}'.format(i, 
                        indices.flatten()[i]+1, similarities.flatten()[i]))
            
    return similarities,indices

In [21]:
similarities,indices = findksimilarusers(1,M, metric='cosine')

4 most similar users for User 1:

1: User 615, with similarity of 0.3332066427480971
2: User 325, with similarity of 0.3177255134631516
3: User 571, with similarity of 0.3130038255306262
4: User 278, with similarity of 0.3074381495210863


In [22]:
similarities,indices = findksimilarusers(1,M, metric='correlation')

4 most similar users for User 1:

1: User 615, with similarity of 0.32497533996702677
2: User 325, with similarity of 0.3096932149159627
3: User 571, with similarity of 0.3048123520589081
4: User 278, with similarity of 0.3001162581206216


In [23]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = metric, k=k):
    prediction=0
    similarities, indices=findksimilarusers(user_id, ratings,metric, k) #similar users based on cosine similarity
    mean_rating = ratings.loc[user_id-1,:].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;
        else: 
            ratings_diff = ratings.iloc[indices.flatten()[i],item_id-1]-np.mean(ratings.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product
    
    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [24]:
predict_userbased(3,4,M);

4 most similar users for User 3:

1: User 233, with similarity of 0.19890091788894193
2: User 274, with similarity of 0.19393248356283055
3: User 125, with similarity of 0.19024241627636895
4: User 458, with similarity of 0.1853865725886743

Predicted rating for user 3 -> item 4: 4


**Item-based Recommendation Systems**

In [25]:
#This function finds k similar items given the item_id and ratings matrix M

def findksimilaritems(item_id, ratings, metric=metric, k=k):
    similarities=[]
    indices=[]    
    ratings=ratings.T
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute')
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.iloc[item_id-1, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    print ('{0} most similar items for item {1}:\n'.format(k,item_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == item_id:
            continue;

        else:
            print ('{0}: Item {1} :, with similarity of {2}'.format(i,indices.flatten()[i]+1, 
                                                                    similarities.flatten()[i]))


    return similarities,indices

In [26]:
similarities,indices=findksimilaritems(3,M)

4 most similar items for item 3:

1: Item 49 :, with similarity of 0.8720912929884415
2: Item 52 :, with similarity of 0.8407979375962897
3: Item 56 :, with similarity of 0.7734692971761392
4: Item 1 :, with similarity of 0.44125958366550966


In [27]:
#This function predicts the rating for specified user-item combination based on item-based approach
def predict_itembased(user_id, item_id, ratings, metric = metric, k=k):
    prediction= wtd_sum =0
    similarities, indices=findksimilaritems(item_id, ratings) #similar users based on correlation coefficients
    sum_wt = np.sum(similarities)-1
    product=1
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == item_id:
            continue;
        else:
            product = ratings.iloc[user_id-1,indices.flatten()[i]] * (similarities[i])
            wtd_sum = wtd_sum + product                              
    prediction = int(round(wtd_sum/sum_wt))
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))      

    return prediction

In [28]:
prediction = predict_itembased(1,3,M)

4 most similar items for item 3:

1: Item 49 :, with similarity of 0.8720912929884415
2: Item 52 :, with similarity of 0.8407979375962897
3: Item 56 :, with similarity of 0.7734692971761392
4: Item 1 :, with similarity of 0.44125958366550966

Predicted rating for user 1 -> item 3: 0


In [38]:
M.tail()

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
996,0.0,3.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
#This function is used to compute adjusted cosine similarity matrix for items
def computeAdjCosSim(M):
    sim_matrix = np.zeros((M.shape[1], M.shape[1]))
    M_u = M.mean(axis=1) #means
          
    for i in range(M.shape[1]):
        for j in range(M.shape[1]):
            if i == j:
                
                sim_matrix[i][j] = 1
            else:                
                if i<j:
                    
                    sum_num = sum_den1 = sum_den2 = 0
                    for k,row in M.loc[:,[i,j]].iterrows(): 

                        if ((M.loc[k,i] != 0) & (M.loc[k,j] != 0)):
                            num = (M[i][k]-M_u[k])*(M[j][k]-M_u[k])
                            den1= (M[i][k]-M_u[k])**2
                            den2= (M[j][k]-M_u[k])**2
                            
                            sum_num = sum_num + num
                            sum_den1 = sum_den1 + den1
                            sum_den2 = sum_den2 + den2
                        
                        else:
                            continue                          
                                       
                    den=(sum_den1**0.5)*(sum_den2**0.5)
                    if den!=0:
                        sim_matrix[i][j] = sum_num/den
                    else:
                        sim_matrix[i][j] = 0


                else:
                    sim_matrix[i][j] = sim_matrix[j][i]           
            
    return pd.DataFrame(sim_matrix)

In [30]:
adjcos_sim = computeAdjCosSim(M)

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


KeyError: 'the label [0] is not in the [index]'

In [31]:
adjcos_sim

NameError: name 'adjcos_sim' is not defined

In [26]:
#This function finds k similar items given the item_id and ratings matrix M

def findksimilaritems_adjcos(item_id, ratings, k=k):
    
    sim_matrix = computeAdjCosSim(ratings)
    similarities = sim_matrix[item_id-1].sort_values(ascending=False)[:k+1].values
    indices = sim_matrix[item_id-1].sort_values(ascending=False)[:k+1].index
    
    print ()'{0} most similar items for item {1}:\n'.format(k,item_id))
    for i in range(0, len(indices)):
            if indices[i]+1 == item_id:
                continue;

            else:
                print ('{0}: Item {1} :, with similarity of {2}'.format(i,indices[i]+1, similarities[i]))
        
    return similarities ,indices    

In [27]:
similarities, indices = findksimilaritems_adjcos(3,M)

4 most similar items for item 3:

1: Item 1 :, with similarity of 0.421262731871
2: Item 6 :, with similarity of -0.0536398904889
3: Item 5 :, with similarity of -0.230521358269
4: Item 4 :, with similarity of -0.767941046575


In [28]:
#This function predicts the rating for specified user-item combination for adjusted cosine item-based approach
#As the adjusted cosine similarities range from -1,+1, sometimes the predicted rating can be negative or greater than max value
#Hack to deal with this: Rating is set to min if prediction is negative, Rating is set to max if prediction is above max
def predict_itembased_adjcos(user_id, item_id, ratings):
    prediction=0

    similarities, indices=findksimilaritems_adjcos(item_id, ratings) #similar users based on correlation coefficients
    sum_wt = np.sum(similarities)-1

    product=1
    wtd_sum = 0 
    for i in range(0, len(indices)):
        if indices[i]+1 == item_id:
            continue;
        else:
            product = ratings.iloc[user_id-1,indices[i]] * (similarities[i])
            wtd_sum = wtd_sum + product                              
    prediction = int(round(wtd_sum/sum_wt))
    if prediction < 0:
        prediction = 1
    elif prediction >10:
        prediction = 10
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))      
        
    return prediction

In [29]:
prediction=predict_itembased_adjcos(3,4,M)

4 most similar items for item 4:

1: Item 2 :, with similarity of 0.0857414341149
2: Item 5 :, with similarity of -0.29905882779
3: Item 1 :, with similarity of -0.519085268895
4: Item 6 :, with similarity of -0.644550286954

Predicted rating for user 3 -> item 4: 6


In [30]:
adjcos_sim

,0,1,2,3,4,5
0,1.000000,0.236908,0.421263,-0.519085,-0.125892,0.010090
1,0.236908,1.000000,-0.805243,0.085741,0.237273,0.520625
2,0.421263,-0.805243,1.000000,-0.767941,-0.230521,-0.053640
3,-0.519085,0.085741,-0.767941,1.000000,-0.299059,-0.644550
4,-0.125892,0.237273,-0.230521,-0.299059,1.000000,0.599158
5,0.010090,0.520625,-0.053640,-0.644550,0.599158,1.000000


In [31]:
#This function utilizes above function to recommend items for selected approach. Recommendations are made if the predicted
#rating for an item is greater than or equal to 6, and the items has not been rated already
def recommendItem(user_id, item_id, ratings):
    
    if user_id<1 or user_id>6 or type(user_id) is not int:
        print ('Userid does not exist. Enter numbers from 1-6')
    else:    
        ids = ['User-based CF (cosine)','User-based CF (correlation)','Item-based CF (cosine)',
               'Item-based CF (adjusted cosine)']

        approach = widgets.Dropdown(options=ids, value=ids[0],
                               description='Select Approach', width='500px')
        
        def on_change(change):
            prediction = 0
            clear_output(wait=True)
            if change['type'] == 'change' and change['name'] == 'value':            
                if (approach.value == 'User-based CF (cosine)'):
                    metric = 'cosine'
                    prediction = predict_userbased(user_id, item_id, ratings, metric)
                elif (approach.value == 'User-based CF (correlation)')  :                       
                    metric = 'correlation'               
                    prediction = predict_userbased(user_id, item_id, ratings, metric)
                elif (approach.value == 'Item-based CF (cosine)'):
                    prediction = predict_itembased(user_id, item_id, ratings)
                else:
                    prediction = predict_itembased_adjcos(user_id,item_id,ratings)

                if ratings[item_id-1][user_id-1] != 0: 
                    print ('Item already rated')
                else:
                    if prediction>=6:
                        print ('\nItem recommended')
                    else:
                        print ('Item not recommended')

        approach.observe(on_change)
        display(approach)

In [32]:
#check for incorrect entries
recommendItem(-1,3,M)

Userid does not exist. Enter numbers from 1-6


In [33]:
recommendItem(3,4,M)

4 most similar users for User 3:

1: User 4, with similarity of 0.90951268934
2: User 2, with similarity of 0.874744414849
3: User 5, with similarity of 0.86545387815
4: User 6, with similarity of 0.853274963344

Predicted rating for user 3 -> item 4: 3
Item not recommended


In [34]:
recommendItem(3,4,M)

4 most similar users for User 3:

1: User 2, with similarity of 0.453897185842
2: User 4, with similarity of 0.451378005098
3: User 6, with similarity of 0.297373304825
4: User 5, with similarity of -0.04288778794

Predicted rating for user 3 -> item 4: 3
Item not recommended


In [35]:
recommendItem(3,4,M)

4 most similar items for item 4:

1: Item 6 :, with similarity of 0.89977997614
2: Item 2 :, with similarity of 0.887160079571
3: Item 5 :, with similarity of 0.88180009273
4: Item 3 :, with similarity of 0.796917800302

Predicted rating for user 3 -> item 4: 6

Item recommended


In [36]:
recommendItem(3,4,M)

4 most similar items for item 4:

1: Item 2 :, with similarity of 0.0857414341149
2: Item 5 :, with similarity of -0.29905882779
3: Item 1 :, with similarity of -0.519085268895
4: Item 6 :, with similarity of -0.644550286954

Predicted rating for user 3 -> item 4: 6

Item recommended


In [37]:
#if the item is already rated, it is not recommended
recommendItem(2,1,M)

4 most similar users for User 2:

1: User 4, with similarity of 0.515910067398
2: User 3, with similarity of 0.453897185842
3: User 6, with similarity of 0.218327934565
4: User 5, with similarity of 0.11245608042

Predicted rating for user 2 -> item 1: 5
Item already rated


In [38]:
#This is a quick way to temporarily suppress stdout in particular code section
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [39]:
#This is final function to evaluate the performance of selected recommendation approach and the metric used here is RMSE
#suppress_stdout function is used to suppress the print outputs of all the functions inside this function. It will only print 
#RMSE values
def evaluateRS(ratings):
    ids = ['User-based CF (cosine)','User-based CF (correlation)','Item-based CF (cosine)','Item-based CF (adjusted cosine)']
    approach = widgets.Dropdown(options=ids, value=ids[0],description='Select Approach', width='500px')
    n_users = ratings.shape[0]
    n_items = ratings.shape[1]
    prediction = np.zeros((n_users, n_items))
    prediction= pd.DataFrame(prediction)
    def on_change(change):
        clear_output(wait=True)
        with suppress_stdout():
            if change['type'] == 'change' and change['name'] == 'value':            
                if (approach.value == 'User-based CF (cosine)'):
                    metric = 'cosine'
                    for i in range(n_users):
                        for j in range(n_items):
                            prediction[i][j] = predict_userbased(i+1, j+1, ratings, metric)
                elif (approach.value == 'User-based CF (correlation)')  :                       
                    metric = 'correlation'               
                    for i in range(n_users):
                        for j in range(n_items):
                            prediction[i][j] = predict_userbased(i+1, j+1, ratings, metric)
                elif (approach.value == 'Item-based CF (cosine)'):
                    for i in range(n_users):
                        for j in range(n_items):
                            prediction[i][j] = predict_userbased(i+1, j+1, ratings)
                else:
                    for i in range(n_users):
                        for j in range(n_items):
                            prediction[i][j] = predict_userbased(i+1, j+1, ratings)
              
        MSE = mean_squared_error(prediction, ratings)
        RMSE = round(sqrt(MSE),3)
        print ("RMSE using {0} approach is: {1}".format(approach.value,RMSE))
              
    approach.observe(on_change)
    display(approach)

In [40]:
evaluateRS(M)

RMSE using Item-based CF (cosine) approach is: 2.804


In [41]:
evaluateRS(M)

RMSE using Item-based CF (cosine) approach is: 2.804


**Thanks for reading this notebook**